### Build aggregate model results from df_pred
Since df_agg is built during model run, it might not have the latest test data applied to it. But we can do this by instead finding loss + acc from df_pred freshly built using the latest test_set, then merge onto df_agg.
 

In [3]:
import json, os, sys
import pandas as pd
sys.path.append('..')
# from modules.trainutils import piece_class_parse
# from modules.trainutils import my_metrics

In [4]:
grid_series = 'expmod2easy'
path_model_analysis = '../models/model-analysis/'
test_path = '../../../other-chess-data/regulation-pieces-1/crops/'

In [5]:
df_agg = pd.read_csv(path_model_analysis + 
                     grid_series +
                     '-df_agg.csv')

df_pred_2 = pd.read_csv(path_model_analysis + 
                         'expmod2' +
                         '-df_pred.csv')

df_pred_easy = pd.read_csv(path_model_analysis + 
                         'expmod2easy' +
                         '-df_pred.csv')

##### Process so that df_pred can give us model_Agg performance

In [6]:
df_agg.sort_values(by = 'test_loss', ascending=True, inplace=True)
df_agg.index = df_agg['Unnamed: 0']
df_agg.index.name = 'modname'
df_agg.drop(columns=['Unnamed: 0'], inplace=True)
df_agg.rename(columns={'test_loss':'reg1_loss', 'test_acc': 'reg1_acc'}, inplace=True)


In [7]:
elems = [('easy', df_pred_easy.copy()),
         ('all2', df_pred_2.copy()),
        ]

for col_name, df_pred in elems:

    mod_names = [col.split('_')[1] for col in df_pred.columns 
                 if 'pred_' in col]

    df_correct = pd.DataFrame([])
    for mod_name in mod_names:

        df_correct[mod_name] = (
          (df_pred['actual'] == 
           df_pred['pred_' + mod_name]).astype('int')
        )
        
    df_loss = pd.DataFrame([])
    for mod_name in mod_names:

        df_loss[mod_name] = df_pred['loss_' + mod_name]
    
        
    N = len(df_correct)
    
    df_acc = pd.DataFrame(
            df_correct.sum(axis=0).map(lambda x: x/ N), 
            columns=[col_name + '_acc'])
    
    df_loss = pd.DataFrame(
            df_loss.sum(axis=0).map(lambda x: x/ N), 
            columns=[col_name + '_loss'])

    df_acc.index = df_acc.index + '.json'
    df_loss.index = df_loss.index + '.json'

    df_agg = df_agg.join(df_acc)
    df_agg = df_agg.join(df_loss)


In [9]:
df_agg.to_csv('../models/model-analysis/expmodreg2all-df_agg.csv')

### Analyze model perf

In [109]:
sort_metric = 'all2_acc' # 'easy_acc'
df_agg.sort_values(by=sort_metric, ascending=False).round(2).T.iloc[-6:,:10]

modname,expmod-b-1.json,expmod-b-10.json,expmod-b-6.json,expmod-b-2.json,expmod-b-27.json,expmod-b-18.json,expmod-b-17.json,expmod-b-14.json,expmod-b-13.json,expmod-b-8.json
reg1_loss,0.68,0.6,0.79,0.9,0.78,0.63,0.72,0.76,0.49,0.67
reg1_acc,0.82,0.85,0.82,0.8,0.8,0.82,0.85,0.77,0.85,0.85
easy_acc,0.94,0.91,0.91,0.93,0.92,0.9,0.93,0.9,0.89,0.88
easy_loss,0.27,0.25,0.28,0.3,0.28,0.36,0.22,0.32,0.29,0.38
all2_acc,0.9,0.88,0.87,0.87,0.87,0.87,0.86,0.85,0.85,0.83
all2_loss,0.45,0.4,0.43,0.52,0.51,0.54,0.43,0.49,0.44,0.58


##### Best all2_acc
 - Generally resize_method= pad or squish
 - why do 17 and 18 perform so well as crop?
 - many bs=16, but equal number of bs=32

In [110]:
sort_metric = 'all2_acc' # 'easy_acc'
df_agg.sort_values(by=sort_metric, ascending=False).round(2).T

modname,expmod-b-1.json,expmod-b-10.json,expmod-b-6.json,expmod-b-2.json,expmod-b-27.json,expmod-b-18.json,expmod-b-17.json,expmod-b-14.json,expmod-b-13.json,expmod-b-8.json,...,expmod-a-25.json,expmod-a-4.json,expmod-a-23.json,expmod-a-26.json,expmod-a-22.json,expmod-a-3.json,expmod-a-24.json,expmod-a-21.json,expmod-b-32.json,expmod-a-32.json
_model_arch,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,...,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50
_fit_one_cycle_epochs,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
_fine_tune_epochs,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
_train_seed,42,42,17,17,17,42,17,42,42,17,...,42,17,17,42,17,42,42,17,42,42
_mult,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
_max_lighting,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
_max_warp,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,...,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4
_max_rotate,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,20,20
_resize_method,pad,pad,squish,pad,pad,crop,crop,pad,pad,pad,...,crop,crop,crop,crop,crop,crop,crop,crop,pad,pad
_pad_mode,zeros,reflection,zeros,zeros,reflection,reflection,reflection,reflection,reflection,reflection,...,reflection,zeros,reflection,reflection,reflection,zeros,reflection,reflection,reflection,reflection


##### Worst on acc2_all
 - Generally crop resize methods
 - p_lighting=.95 (instead of .9)
 - bs <= 16

In [111]:
sort_metric = 'all2_acc' # 'easy_acc'
df_agg.sort_values(by=sort_metric, ascending=True).round(2).T.iloc[:,:]

modname,expmod-a-21.json,expmod-a-24.json,expmod-a-3.json,expmod-a-22.json,expmod-a-26.json,expmod-a-23.json,expmod-a-4.json,expmod-a-25.json,expmod-b-25.json,expmod-a-2.json,...,expmod-b-14.json,expmod-b-17.json,expmod-b-2.json,expmod-b-18.json,expmod-b-27.json,expmod-b-6.json,expmod-b-10.json,expmod-b-1.json,expmod-b-32.json,expmod-a-32.json
_model_arch,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,...,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50
_fit_one_cycle_epochs,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
_fine_tune_epochs,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
_train_seed,17,42,42,17,42,17,17,42,42,17,...,42,17,17,42,17,17,42,42,42,42
_mult,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
_max_lighting,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
_max_warp,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,...,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4
_max_rotate,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,20,20
_resize_method,crop,crop,crop,crop,crop,crop,crop,crop,crop,pad,...,pad,crop,pad,crop,pad,squish,pad,pad,pad,pad
_pad_mode,reflection,reflection,zeros,reflection,reflection,reflection,zeros,reflection,reflection,zeros,...,reflection,reflection,zeros,reflection,reflection,zeros,reflection,zeros,reflection,reflection


##### Best on test_loss

In [116]:
sort_metric = 'all2_loss' # 'easy_acc'
df_agg.sort_values(by=sort_metric, ascending=True).round(2).T.iloc[:,:]

modname,expmod-b-10.json,expmod-b-6.json,expmod-b-17.json,expmod-b-13.json,expmod-b-1.json,expmod-b-14.json,expmod-b-27.json,expmod-b-2.json,expmod-b-12.json,expmod-b-18.json,...,expmod-a-23.json,expmod-a-25.json,expmod-a-4.json,expmod-a-26.json,expmod-a-22.json,expmod-a-24.json,expmod-a-3.json,expmod-a-21.json,expmod-b-32.json,expmod-a-32.json
_model_arch,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,...,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50
_fit_one_cycle_epochs,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
_fine_tune_epochs,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
_train_seed,42,17,17,42,42,42,17,17,42,42,...,17,42,17,42,17,42,42,17,42,42
_mult,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
_max_lighting,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
_max_warp,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,...,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4
_max_rotate,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,20,20
_resize_method,pad,squish,crop,pad,pad,pad,pad,pad,pad,crop,...,crop,crop,crop,crop,crop,crop,crop,crop,pad,pad
_pad_mode,reflection,zeros,reflection,reflection,zeros,reflection,reflection,zeros,reflection,reflection,...,reflection,reflection,zeros,reflection,reflection,reflection,zeros,reflection,reflection,reflection


In [115]:
sort_metric = 'easy_loss' # 'easy_acc'
df_agg.sort_values(by=sort_metric, ascending=True).round(2).T.iloc[:,:]

modname,expmod-b-17.json,expmod-b-10.json,expmod-b-1.json,expmod-b-27.json,expmod-b-6.json,expmod-b-13.json,expmod-b-32.json,expmod-b-2.json,expmod-b-20.json,expmod-b-14.json,...,expmod-b-3.json,expmod-b-25.json,expmod-a-23.json,expmod-a-25.json,expmod-a-22.json,expmod-a-4.json,expmod-a-26.json,expmod-a-3.json,expmod-a-24.json,expmod-a-21.json
_model_arch,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,...,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50,resnet50
_fit_one_cycle_epochs,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
_fine_tune_epochs,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
_train_seed,17,42,42,17,17,42,42,17,42,42,...,42,42,17,42,17,17,42,42,42,17
_mult,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
_max_lighting,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,...,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9,0.9
_max_warp,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,...,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.4
_max_rotate,20,20,20,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,20,20
_resize_method,crop,pad,pad,pad,squish,pad,pad,pad,crop,pad,...,crop,crop,crop,crop,crop,crop,crop,crop,crop,crop
_pad_mode,reflection,reflection,zeros,reflection,zeros,reflection,reflection,zeros,reflection,reflection,...,zeros,reflection,reflection,reflection,reflection,zeros,reflection,zeros,reflection,reflection
